In [2]:
import pandas as pd
import re
from nltk.stem import wordnet, WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords
from nltk import pos_tag
from tqdm import tqdm_notebook as tqdm
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer

## READ DATASET AND CREATE TEXT AND TARGET

In [3]:
dataset = pd.read_json('data/News_Category_Dataset_v2.json', lines=True)

# keep only text and target category
dataset = dataset.drop(['date', 'link', 'authors'], axis=1)
dataset.head(10)

,category,headline,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ..."
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,"""It is not right to equate horrific incidents ..."
6,ENTERTAINMENT,Donald Trump Is Lovin' New McDonald's Jingle I...,"It's catchy, all right."
7,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This ...,There's a great mini-series joining this week.
8,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,"Myer's kids may be pushing for a new ""Powers"" ..."
9,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,You're getting a recent Academy Award-winning ...


In [4]:
target = (dataset['category'].map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)).tolist()

In [5]:
target[0:4]

['CRIME', 'ENTERTAINMENT', 'ENTERTAINMENT', 'ENTERTAINMENT']

In [6]:
texts = (dataset['headline'] + ' ' + dataset['short_description']).tolist()

In [7]:
texts[0:4]

['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV She left her husband. He killed their children. Just another day in America.',
 "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song Of course it has a song.",
 'Hugh Grant Marries For The First Time At Age 57 The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.',
 "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump."]

## TEXT CLEANING

In [ ]:
patternAlphaNum = re.compile('[\W_]+')
stops = stopwords.words('english')
wn = WordNetLemmatizer()
stemmer = SnowballStemmer('english')

# helper function to transform pos-tags
# this is ridiculous btw
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n'

def clean_text(text, lemma_or_stem='lemma', word_transformer=wn, pattern=patternAlphaNum, stops=stops):
    # split to words and lowercase
    words = [word.lower() for word in text.split()]
    
    # keep only alphanumeric characters
    words_alphanum = [pattern.sub('', word) for word in words]
    
    # remove stopwords
    words_notstop = [word for word in words_alphanum if not ((word in stops) or (word == ''))]
    
    # lemmatize using pos tags
    if lemma_or_stem == "lemma":
        pos_tags = [get_wordnet_pos(pos[1]) for pos in pos_tag(words_notstop)]
        return ' '.join([word_transformer.lemmatize(word, pos=pos_tag) for word, pos_tag in zip(words_notstop, pos_tags)])
    
    # stemming
    elif lemma_or_stem == "stem":
        return ' '.join([word_transformer.stem(word) for word in words_notstop])
    
    # return raw words
    else:
        return ' '.join(words_notstop)
        

In [ ]:
texts_cleaned_lemma = [clean_text(text, 'lemma', wn) for text in tqdm(texts)]
texts_cleaned_stem = [clean_text(text, 'stem', stemmer) for text in tqdm(texts)]

In [ ]:
# combine them to 1 text
all_cleaned_text_lemma = '\n'.join(texts_cleaned_lemma)
all_cleaned_text_stem = '\n'.join(texts_cleaned_stem)

# write to file
with open('cleaned_text/cleaned_text_lemma.txt', 'wb') as f:
    f.write(all_cleaned_text_lemma.encode('utf-8'))
with open('cleaned_text/cleaned_text_stem.txt', 'wb') as f:
    f.write(all_cleaned_text_stem.encode('utf-8'))

## FEATURE ENGINEERING

In [1]:
# load text
with open('cleaned_text/cleaned_text_stem.txt', 'rb') as f:
    all_cleaned_text_stem = [line.decode('utf-8').strip('\n') for line in f.readlines()]
with open('cleaned_text/cleaned_text_lemma.txt', 'rb') as f:
    all_cleaned_text_lemma = [line.decode('utf-8').strip('\n') for line in f.readlines()]

In [9]:
# metadata
blobs = [TextBlob(text) for text in texts]
polarities = [blob.polarity for blob in blobs]
subjectivities = [blob.subjectivity for blob in blobs]
lens = [len(text) for text in texts]

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
tfidf_unigram = TfidfVectorizer()

In [ ]:
text_vectors_unigram = tfidf_unigram.fit_transform(all_cleaned_text)

In [ ]:
text_vectors_unigram.shape

In [ ]:
tfidf_unigram.vocabulary_.keys()

## FEATURE UNION